In [5]:
pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd


In [3]:
df_SO2_perX = pd.read_csv('CSV_data\GEODAIR\O3\O3_moy_hor.per8.csv', sep = ';')

df_SO2_perX.describe()

,valeur,valeur brute,taux de saisie,couverture temporelle,couverture de données,validité,Latitude,Longitude
count,54915.000000,54915.000000,54915.000000,54915.0,54915.000000,54915.000000,54915.000000,54915.000000
mean,77.591960,77.578829,98.523300,100.0,98.523300,0.965437,43.921851,2.385489
std,24.053999,24.054031,7.440464,0.0,7.440464,0.260637,12.356722,14.251810
min,0.100000,0.100000,4.000000,100.0,4.000000,-1.000000,-21.334406,-63.081481
25%,64.300000,64.275000,100.000000,100.0,100.000000,1.000000,44.060347,1.047892
50%,75.700000,75.725000,100.000000,100.0,100.000000,1.000000,46.652668,3.058333
75%,89.300000,89.275000,100.000000,100.0,100.000000,1.000000,48.729694,5.394716
max,206.300000,206.275000,100.000000,100.0,100.000000,1.000000,51.032761,55.582620


### On combine les dataframes dans leur longueure

Les data frames sont chacuns divisisés en 8 périodes en raison des limitations liées à la méthode d'importation des données. On concatène donc les données liées à chaque polluant dans la longueur en procédant de la dernière période à la première (chaque période représentant une période de 6 mois classé dans l'odre antichronologique au sein de chaque data frame).

In [10]:
folder_paths = [
   r"CSV_data\GEODAIR\NO2\NO2_moy_hor_per", #Utiliser soit r"..." soit des \\ sinon les "\N" de "\NO2" posent problème
    "CSV_data\GEODAIR\O3\O3_moy_hor_per",
    "CSV_data\GEODAIR\PM2.5\PM2.5_moy_jour_per",
    "CSV_data\GEODAIR\PM10\PM10_moy_jour_per",
    "CSV_data\GEODAIR\SO2\SO2_moy_hor_per"
    ]

#Ici l'instruction "range(8,1,-1)" permet de classer les chemins du plus récent au moins récent.
all_file_paths = [[ folder + str(i) + ".csv" for i in range(8,0,-1)] for folder in folder_paths]


all_combined_df = [pd.concat([pd.read_csv(file, sep = ";") for file in all_file_paths[i]]) for i in range(5)]

longeurs = [len(df) for df in all_combined_df]
longeurs
#Fonctionne
# Output : [523418, 437621, 306241, 528658, 129022] le df le plus long est celui des mesures de NO2

[523418, 437621, 306241, 528658, 129022]

### Jointures

Pour calculer les indices de polluants localement, le plus simple est de joindre les data frames sur la clé suivante : [date, localisation]. On commence par joindre les différents dataframes sur le plus petits d'entre eux afin de maximiser la complétude des champs de données.

In [15]:
all_combined_df[3].head(2)


,Date de début,Date de fin,Organisme,code zas,Zas,code site,nom site,type d'implantation,Polluant,type d'influence,...,valeur,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,Latitude,Longitude
0,2023/07/01 00:00:00,2023/07/01 23:59:59,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,PM10,Fond,...,6.0,5.875000,µg-m3,100.0,100,100.0,A,1,49.119442,6.180833
1,2023/07/01 00:00:00,2023/07/01 23:59:59,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01012,Metz-Borny,Urbaine,PM10,Fond,...,7.0,6.645833,µg-m3,100.0,100,100.0,A,1,49.110283,6.223336


In [20]:

# Specify the columns to join on
join_columns = ["Date de début", "Date de fin", "code site"]

# Find the smallest dataframe based on the number of rows
smallest_df = min(all_combined_df, key=len)

# Initialize the result dataframe with the smallest dataframe
result_df = smallest_df

# Merge each dataframe in the list with unique suffixes
for i, df in enumerate(all_combined_df):
    if df is not smallest_df:  # Avoid self-joining
        result_df = pd.merge(
            result_df, 
            df, 
            on=join_columns, 
            how='inner', 
            suffixes=(None, f'_df{i}')  # Unique suffix for each dataframe
        )

# result_df now contains the inner join of all dataframes on the specified columns


In [27]:
columns_list = result_df.columns.tolist()

print(columns_list, len(columns_list))

['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'Réglementaire', "type d'évaluation", 'type de valeur', 'valeur', 'valeur brute', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité', 'Latitude', 'Longitude', 'Organisme_df0', 'code zas_df0', 'Zas_df0', 'nom site_df0', "type d'implantation_df0", 'Polluant_df0', "type d'influence_df0", 'Réglementaire_df0', "type d'évaluation_df0", 'type de valeur_df0', 'valeur_df0', 'valeur brute_df0', 'unité de mesure_df0', 'taux de saisie_df0', 'couverture temporelle_df0', 'couverture de données_df0', 'code qualité_df0', 'validité_df0', 'Latitude_df0', 'Longitude_df0', 'Organisme_df1', 'code zas_df1', 'Zas_df1', 'nom site_df1', "type d'implantation_df1", 'Polluant_df1', "type d'influence_df1", 'Réglementaire_df1', "type d'évaluation_df1", 'type de valeur_df1', 'valeur_df1', 'valeur brute_df1', 'un